# Importing Libraries

In [2]:
import requests
import urllib
import urllib.request
import re
import os
import csv
import smtplib
import os.path
from bs4 import BeautifulSoup
import csv
import pysal as ps
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.cm as cm
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
import tabula
import PyPDF2
from datetime import timedelta, date
import pickle
import os.path
import csv
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

C:\Users\Wan Xin\AppData\Local\ESRI\conda\envs\ge5219\lib\site-packages\pysal\explore\segregation\network\network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
C:\Users\Wan Xin\AppData\Local\ESRI\conda\envs\ge5219\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


# Scraping Data using Beautiful Soup
you can find a tutorial on web scraping data tables from Wiki using bs here: 
https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [3]:
# Start by parsing my URL
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

In [4]:
# Finding the table
My_table = soup.find('table',{'class':'wikitable sortable'})

<p> I will now take all the texts from My_table. In order for '\n' (which is a html formatting that will also get stripped, i have to use 'get_text(strip=True)' </p>

In [5]:
table_contents = []
for i in My_table.find_all('td'):
    table_contents.append(i.get_text(strip=True))

In [6]:
# stripping every third item in the contents starting from the appropriae index
postcode = [table_contents[i] for i in range(0, len(table_contents), 3)]
borough = [table_contents[i] for i in range(1, len(table_contents), 3)]
neigh = [table_contents[i] for i in range(2, len(table_contents), 3)]

In [7]:
# Creating my dataframe
d = {'Postcode': postcode, 'Borough': borough, 'Neighbourhood': neigh}
df = pd.DataFrame(data=d)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Processing the Dataframe
**<p> To process the dataframe, we have to do the following: </p>**
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


**<p> Let's start with the first objective! ignoring cells with borough = Not assigned </p>**

In [8]:
df = df[~df['Borough'].str.contains("Not assigned")]
df.reset_index(drop=True).head() 

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


**<p> Let's move on, now to make a borough appear only once in the borough column, and join the neighbourhoods in the neighbourhoods column separated by a comma </p>**
<p> Here is a helpful link: https://stackoverflow.com/questions/52546386/pandas-groupby-multiple-columns-concatenating-one-column-while-adding-another </p>

In [9]:
# passing on to another variable to not overwrite
df_sorted = df.groupby(['Postcode', 'Borough'], as_index=False).agg({'Neighbourhood':lambda x: ', '.join(x)})
df_sorted.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**<p> Last step! If Neighbourhood = Not assigned, Neighbourhood = Borough</p>**

In [11]:
for i in range(len(df_sorted)):
    if 'Not assigned' in df_sorted['Neighbourhood'][i]:
        df_sorted['Neighbourhood'][i] = df_sorted['Neighbourhood'][i].replace('Not assigned', df_sorted['Borough'][i])
        
print(display(df_sorted.head()))

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


None


**<p> Yay! Now let's see the shape of our final dataframe </p>**

In [12]:
row, column = df_sorted.shape

In [103]:
print('the final dataframe has ' + str(row) + ' rows, and ' + str(column) + ' columns')

the final dataframe has 103 rows, and 3 columns


In [115]:
df_sorted.to_csv (r'C:\Users\Wan Xin\Documents\IBM coursera\toronto_neigh.csv', index = None, header=True)